In [1]:
import warnings
warnings.filterwarnings("ignore")

import tensorflow as tf
import pandas as pd
import whisper
from moviepy.editor import *

In [4]:
from google_drive_downloader import GoogleDriveDownloader as gdd
# gdd.DOWNLOAD_URL()/
video_file = gdd.download_file_from_google_drive(file_id="1QkQNI8tAZWGQybCy69dTmZ7RXvB4eQkS",
        dest_path='./data/video.mp4', unzip=False, overwrite=True)

In [ ]:
name_person = "./data/video"

In [ ]:
# tiny, base, small, medium, large: pick one model among these
model = whisper.load_model("small")

video_file_name = name_person + '.mp4'
audio_file_name = name_person + '_audio.mp3'

read_video_file = True
write_audio_file = True

In [ ]:
if read_video_file:
    video = VideoFileClip(video_file_name)
    audio = video.audio
    if write_audio_file:
        audio.write_audiofile(audio_file_name, )

In [ ]:
output = model.transcribe(name_person + "_audio.mp3", language='en')
transcription_df = pd.json_normalize(output['segments'])

transcript = output['text']

In [ ]:
transcription_df['temp'] = "{"+ transcription_df['start'].astype(str) + ", " + transcription_df['end'].astype(str) +"}" + transcription_df['text'] 

transcription1 = '\n'.join(transcription_df['temp'].to_list())

transcription2 = "\n".join(transcription_df['text'])

In [ ]:
save_complete_text = True
if save_complete_text:
    with open(name_person + '_transcript1.txt', 'w') as f:
        f.write(transcription1)
    with open(name_person + '_transcript2.txt', 'w') as f:
        f.write(transcription2)
